In [ ]:
import panel as pn
pn.extension('tabulator', notifications=True)

import panel as pn
import matplotlib.pyplot as plt
from src.dao import produtor_dao, plantio_dao, propriedade_dao

In [ ]:
pn.extension('tabulator', notifications=True)

# --- WIDGETS (CAMPOS DA TELA) ---
# ID Oculto (para saber se é edição)
w_id = pn.widgets.IntInput(name="ID", disabled=True, visible=False)

w_nome = pn.widgets.TextInput(name="Nome Completo", placeholder="Ex: João da Silva")
w_cpf = pn.widgets.TextInput(name="CPF", placeholder="12345678900")
w_nasc = pn.widgets.DatePicker(name="Data de Nascimento")
w_insc = pn.widgets.TextInput(name="Inscrição Estadual", placeholder="Ex: 200500-1")
w_busca = pn.widgets.TextInput(name="🔍 Buscar Produtor", placeholder="Digite o nome e dê Enter")

# Botões
btn_salvar = pn.widgets.Button(name='💾 Salvar', button_type='success')
btn_limpar = pn.widgets.Button(name='🧹 Limpar', button_type='default')
btn_excluir = pn.widgets.Button(name='🗑️ Excluir', button_type='danger', disabled=True)

# Tabela
tabela = pn.widgets.Tabulator(show_index=False, sizing_mode='stretch_width', page_size=10)

# --- LÓGICA (EVENTOS) ---

async def carregar_tabela(event=None):
    """Busca dados no banco e atualiza a tabela"""
    df = await produtor_dao.listar(w_busca.value)
    tabela.value = df
    # Atualiza gráfico também
    await atualizar_grafico()

def preencher_formulario(event):
    """Ao clicar na tabela, joga os dados para os campos"""
    if event.row is not None:
        row = event.row
        w_id.value = int(row['id_pessoa'])
        w_nome.value = str(row['nome'])
        w_cpf.value = str(row['cpf'])
        w_insc.value = str(row['inscricao_estadual'])
        if row['data_nascimento']:
            w_nasc.value = row['data_nascimento']
        
        # Muda estado dos botões para "Modo Edição"
        btn_salvar.name = "🔄 Atualizar"
        btn_salvar.button_type = "warning"
        btn_excluir.disabled = False

tabela.on_click(preencher_formulario)

async def salvar_clique(event):
    try:
        btn_salvar.loading = True
        
        # Validação simples
        if not w_cpf.value or not w_nome.value:
            pn.state.notifications.error("Nome e CPF são obrigatórios!")
            return

        if w_id.value: # TEM ID? ENTÃO É ATUALIZAÇÃO
            await produtor_dao.atualizar(w_id.value, w_nome.value, w_cpf.value, w_nasc.value, w_insc.value)
            pn.state.notifications.success("Produtor atualizado!")
        else: # NÃO TEM ID? ENTÃO É INSERÇÃO
            await produtor_dao.inserir(w_nome.value, w_cpf.value, w_nasc.value, w_insc.value)
            pn.state.notifications.success("Produtor cadastrado!")
        
        limpar_clique(None) # Reseta formulário
        await carregar_tabela() # Recarrega lista
        
    except Exception as e:
        pn.state.notifications.error(f"Erro: {str(e)}")
    finally:
        btn_salvar.loading = False

async def excluir_clique(event):
    if w_id.value:
        try:
            await produtor_dao.excluir(w_id.value)
            pn.state.notifications.success("Produtor excluído!")
            limpar_clique(None)
            await carregar_tabela()
        except Exception as e:
            pn.state.notifications.error(f"Erro ao excluir: {e}")

def limpar_clique(event):
    w_id.value = 0 # Reseta ID
    w_nome.value = ""
    w_cpf.value = ""
    w_insc.value = ""
    w_nasc.value = None
    
    # Reseta botões
    btn_salvar.name = "💾 Salvar"
    btn_salvar.button_type = "success"
    btn_excluir.disabled = True

# Ligar botões às funções
btn_salvar.on_click(salvar_clique)
btn_limpar.on_click(limpar_clique)
btn_excluir.on_click(excluir_clique)
w_busca.param.watch(carregar_tabela, 'value') # Busca ao digitar

# --- GRÁFICO ---
pane_grafico = pn.pane.Matplotlib(tight=True)

async def atualizar_grafico():
    df = await produtor_dao.dados_grafico_idade()
    if df.empty: return
    
    fig, ax = plt.subplots(figsize=(5, 3))
    ax.bar(df['faixa_etaria'], df['total'], color='#4CAF50')
    ax.set_title("Produtores por Faixa Etária")
    ax.set_ylabel("Quantidade")
    plt.close(fig)
    pane_grafico.object = fig

# --- LAYOUT FINAL ---
pn.state.onload(carregar_tabela) # Carrega ao abrir

app = pn.Column(
    pn.pane.Markdown("# 🚜 Módulo Produtor"),
    pn.Row(
        pn.Column(
            pn.pane.Markdown("### Cadastro"),
            w_id, w_nome, w_cpf, w_nasc, w_insc,
            pn.Row(btn_salvar, btn_limpar, btn_excluir)
        ),
        pn.Column(
            pn.pane.Markdown("### Listagem"),
            w_busca,
            tabela,
            pn.pane.Markdown("### Estatísticas"),
            pane_grafico
        )
    )
)

app.servable() 
